**Install python dependencies**

In [ ]:
!pip install -q -r ./dependencies/requirements.txt

**Load python libraries**

In [ ]:
from sqlalchemy import create_engine
from os import getenv
import pandas as pd
from datetime import datetime
import smart_match
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

**Connect to database**

In [ ]:
db_name = getenv("POSTGRES_DB")
db_user = getenv("POSTGRES_USER")
db_pass = getenv("POSTGRES_PASSWORD")
db_host = 'localhost'
db_port = '5432'
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

**Preprocess Data: Event Data**

In [ ]:
# getting event data from database
db_response = db.execute(
    f"""select  (coalesce(p.first_name, '') || ' ' ||coalesce(p.last_name, '')) as name,
       position,
       club_id,
       number as matchday,
       type as event_type,
       count(*) from events
        inner join matchdays m on events.matchday_id = m.id
        inner join players p on events.player_id = p.id
    where corrected is false and p.active is true
    group by name, position, matchday, event_type, club_id
    order by name, matchday,  event_type
    """
)

df_event = pd.DataFrame(db_response.fetchall())
df_event.columns = db_response.keys()

df_event

In [ ]:
# load event_types
event_types = ['pass', 'unsuccessfulPass', 'superPass', 'throwIn', 'goalAssist', 'farCorner', 'cross', 'freeKick', 'interception', 'goalMissedFar', 'shotAtGoal', 'unsuccessfulPenalty', 'missedChance', 'goal', 'doublePack', 'hattrick', 'penaltyGoal', 'ownGoal', 'foul','awardedPenalty', 'causedPenalty', 'yellowCard', 'secondYellowCard', 'redCard', 'successfulTackle', 'unsuccessfulTackle', 'blockedGoalShot', 'offside', 'error', 'savedPenalty', 'lostPenalty', 'defended', 'goalAgainst']

In [ ]:
df_players = df_event.groupby('name')

df_final = pd.DataFrame(columns=['name','position','matchday','event_type','count', 'club_id'])

# add 0 values
for player_tuple in df_players:
    player_name = player_tuple[0]
    position = player_tuple[1]['position'].max()
    club_id = player_tuple[1]['club_id'].max()

    df_player = player_tuple[1]

    for matchday in range(1,35):
        for event_type in event_types:
            if df_player.loc[(df_player['matchday'] == matchday) & (df_player['event_type'] == event_type)].empty == True:
                df_player = df_player.append({
                    'name': player_name, 
                    'position': position,
                    'club_id': club_id,
                    'matchday': matchday,
                    'event_type': event_type,
                    'count': 0
                }, ignore_index=True)
    
    df_final = df_final.append(df_player)

df_final.to_csv('./data/events_prep2.csv', index=False)

**Preprocess Data: Betting Odds**

In [ ]:
# load odds from csv
df_odds_csv = pd.read_csv('./data/odds_20_21.csv')
df_odds = df_odds_csv[['Date', 'HomeTeam', 'AwayTeam', 'AvgH', 'AvgD', 'AvgA']]
df_odds['Date'] = df_odds['Date'].map(lambda x: datetime.strptime(datetime.strptime(x, "%d/%m/%Y").strftime("%Y-%m-%d"), "%Y-%m-%d"))
df_odds['matchday'] = 0

# add matchday to odds
df_matchdays_db = pd.read_sql_table('matchdays', db)
df_matchdays = df_matchdays_db[['start', 'end', 'number']]
df_matchdays['start'] = df_matchdays['start'].map(lambda x: datetime.strptime(x.split('T')[0], "%Y-%m-%d"))
df_matchdays['end'] = df_matchdays['end'].map(lambda x: datetime.strptime(x.split('T')[0], "%Y-%m-%d"))

for index, row in df_odds.iterrows():
    for index2, row2 in df_matchdays.iterrows():
        if row2['start'] <= row['Date'] <= row2['end']:
            df_odds.loc[index, 'matchday'] = row2['number']

In [ ]:
# game odds to win, lose, draw odds per team
df_odds_team = pd.DataFrame(columns=['team_name','matchday', 'is_home', 'odds_win','odds_draw','odds_lose'])

for index, row in df_odds.iterrows():
    df_odds_team = df_odds_team.append({
        'team_name': row['HomeTeam'],
        'matchday': row['matchday'],
        'is_home': True,
        'odds_win': row['AvgH'],
        'odds_draw': row['AvgD'],
        'odds_lose': row['AvgA'],
    } , ignore_index=True)

    df_odds_team = df_odds_team.append({
        'team_name': row['AwayTeam'],
        'matchday': row['matchday'],
        'is_home': False,
        'odds_win': row['AvgA'],
        'odds_draw': row['AvgD'],
        'odds_lose': row['AvgH'],
    } , ignore_index=True)

In [ ]:
# odds team names to club_ids
df_teams = pd.read_sql_table('teams', db)

# default is levenstein, which mixes Augsburg and Freiburg
smart_match.use('Smith Waterman')

def name_to_club_id(name): 
    most_similar = {'db_club_id': '', 'similarity': 0 }
    for index, db_team in df_teams.iterrows():
        similarity = smart_match.similarity(db_team['name'], name)
        if most_similar['similarity'] < similarity:
            most_similar['db_club_id'] = db_team['id']
            most_similar['similarity'] = similarity
            
    return most_similar['db_club_id']

df_odds_team['club_id'] = df_odds_team['team_name'].map(lambda x: name_to_club_id(x))

df_odds_team = df_odds_team.sort_values(by=['matchday', 'club_id'])

df_odds_team.to_csv('./data/odds_prep.csv', index=False)

# !!! MANUALLY CHANGE COVID SHIFTED MATCHES !!!

In [ ]:
# DEBUGGING CELL

# df_teams = pd.read_sql_table('teams', db)

# smart_match.use('Smith Waterman')

# df_similarities = pd.DataFrame(columns=['odd_name','db_name', 'similarity'])

# for team_tuple in df_odds_team.groupby("team_name"):
#     odd_name = team_tuple[0]
    
#     for index, row in df_teams.iterrows():
#         db_name = row['name']
#         df_similarities = df_similarities.append({'odd_name': odd_name, 'db_name': db_name, 'similarity': smart_match.similarity(odd_name, db_name)}, ignore_index=True)

# # df_similarities.to_csv('./data/similarities.csv')

# df_similarities[df_similarities.groupby("odd_name")['similarity'].transform(max) == df_similarities['similarity']]

**Merge Event data and Betting Odds**

In [ ]:
df_events = pd.read_csv('./data/events_prep.csv')
df_odds = pd.read_csv('./data/odds_prep.csv')

df = pd.merge(df_events, df_odds)

# remove playeron and playeroff
df = df.drop(df[df.event_type == 'playerOn'].index)
df = df.drop(df[df.event_type == 'playerOff'].index)

# rename count to occurences
df = df.rename(columns={'count':'occurences'})

df.to_csv('./data/data.csv', index=False)

**Create Label/Target Column**

The label for each row is the upcoming value.

In [ ]:
df = pd.read_csv('./data/data.csv')

df_label = pd.DataFrame()

for event_type, df_e in tqdm(df.groupby('event_type')):
    for player_name, df_e_p in df_e.groupby('name'):
        
        df_e_p = df_e_p.sort_values('matchday')

        # Add weighted occurences
        df_e_p.set_index('matchday', inplace=True)
        df_e_p['weighted_occurences'] = df_e_p.occurences.ewm(alpha=0.5, adjust=False).mean().map(lambda x: int(x))

        # Add label
        df_e_p['label'] = df_e_p['occurences'].shift(periods=-1, fill_value=0)

        df_label = df_label.append(df_e_p)

df_label.to_csv('./data/data_labeled.csv')

df_label

**Split dataframe into input and label; transform categorical values into numbers**

In [11]:
df_label = pd.read_csv('./data/data_labeled.csv')

label = df_label['label']
df_inputs = df_label.drop('label', axis='columns')

df_inputs['team_name_normalized'] = LabelEncoder().fit_transform(df_inputs['team_name'])
df_inputs['event_type_normalized'] = LabelEncoder().fit_transform(df_inputs['event_type'])
df_inputs['position_normalized'] = LabelEncoder().fit_transform(df_inputs['position'])

df_inputs_n = df_inputs.drop(['team_name', 'event_type', 'position', 'club_id', 'occurences', 'name'], axis='columns')

label.to_csv('./data/label.csv')
df_inputs_n.to_csv('./data/inputs.csv')